# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import requests
import xlsxwriter
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': '(NTMENSO AQELSB RADLGTLA/K)E GCSA', 'calculationPrice': 'close', 'open': 136, 'openTime': 1638531652131, 'openSource': 'oiiaflcf', 'close': 133.83, 'closeTime': 1640360695777, 'closeSource': 'iaoflifc', 'high': 136.59, 'highTime': 1647345026177, 'highSource': ' tudr5eiec yle npda1iem', 'low': 134.46, 'lowTime': 1621253693570, 'lowSource': ' lecr 5tpee1diaunyem id', 'latestPrice': 137.92, 'latestSource': 'Close', 'latestTime': 'December 31, 2020', 'latestUpdate': 1642003006206, 'latestVolume': 99507871, 'iexRealtimePrice': 138.7, 'iexRealtimeSize': 1, 'iexLastUpdated': 1681545273894, 'delayedPrice': 133.84, 'delayedPriceTime': 1656864657680, 'oddLotDelayedPrice': 139.34, 'oddLotDelayedPriceTime': 1626893002849, 'extendedPrice': 135.48, 'extendedChange': 0.05, 'extendedChangePercent': 0.0004, 'extendedPriceTime': 1665938325152, 'previousClose': 137.19, 'previousVolume': 100951234, 'change': -1.05, 'changePercent': -0.008,

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
peratio = data["peRatio"]
peratio

40.81

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    r = requests.get(batch_api_call_url)
    if r.status_code < 200 or r.status_code >= 300:
        raise RuntimeError (r.reason)
    data = r.json()
    for key, value in  data.items():
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    key,
                    value["quote"]["latestPrice"],
                    value["quote"]["peRatio"],
                    "N/A"
                ],
                index = my_columns
            ),
            ignore_index = True
        )
        

## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
sort_column = my_columns[2]
final_dataframe.sort_values(sort_column, ascending = True, inplace=True)
final_dataframe = final_dataframe[final_dataframe[sort_column]>0]
final_dataframe = final_dataframe[:50].reset_index()
final_dataframe.drop(["index"], axis=1, inplace=True)

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
        return val
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_size = portfolio_input()

Enter the value of your portfolio:1000000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = portfolio_size / len(final_dataframe)
for cnt in range(len(final_dataframe)):
    final_dataframe.iloc[cnt, -1] = math.floor(position_size / final_dataframe.iloc[cnt, 1])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,38.25,2.28,522875
1,NLOK,21.13,4.07,946521
2,BIO,594.34,5.07,33650
3,UNM,23.19,5.13,862440
4,AIV,5.47,5.22,3656307
5,KIM,15.59,6.71,1282873
6,MET,47.06,7.8,424989
7,ALL,110.17,7.9,181537
8,EBAY,51.29,8.13,389939
9,BIIB,256.14,8.23,78082


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = "AAPL"
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats,financials&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url)
print(data.status_code)

data = data.json()
# Price-to-earnings ratio
peratio = data[symbol]["quote"]["peRatio"]
# Price-to-book ratio
pbratio = data[symbol]["advanced-stats"]["priceToBook"]
# Price-to-sales ratio
psratio = data[symbol]["advanced-stats"]["priceToSales"]
# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]["advanced-stats"]["enterpriseValue"]
ebitda = data[symbol]["advanced-stats"]["EBITDA"]
ev_to_ebitda = enterprise_value / ebitda
# Enterprise Value divided by Gross Profit (EV/GP)
grossProfit = data[symbol]["advanced-stats"]["grossProfit"]
ev_to_gp = enterprise_value / grossProfit

200


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]
rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats,financials&token={IEX_CLOUD_API_TOKEN}'
    r = requests.get(batch_api_call_url)
    if r.status_code < 200 or r.status_code >= 300:
        raise RuntimeError (r.reason)
    data = r.json()
    for key, value in  data.items():
        enterprise_value = value['advanced-stats']['enterpriseValue']
        ebitda = value['advanced-stats']['EBITDA']
        gross_profit = value['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        rv_dataframe = rv_dataframe.append(
            pd.Series(
                [
                    key,
                    value['quote']['latestPrice'],
                    'N/A',
                    value['quote']['peRatio'],
                    'N/A',
                    value['advanced-stats']['priceToBook'],
                    'N/A',
                    value['advanced-stats']['priceToSales'],
                    'N/A',
                    ev_to_ebitda,
                    'N/A',
                    ev_to_gross_profit,
                    'N/A',
                    'N/A'
                ],
                index = rv_columns
            ),
            ignore_index = True
        )


In [13]:
rv_dataframe.head()

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,123.96,N/A,55.78,N/A,7.5,N/A,7.11,N/A,33.513979,N/A,13.356296,N/A,N/A
1,AAL,16.46,N/A,-1.15,N/A,-1.73,N/A,0.3957,N/A,-6.101125,N/A,1.398429,N/A,N/A
2,AAP,159.88,N/A,23.1,N/A,2.9,N/A,1.08,N/A,10.958001,N/A,2.510845,N/A,N/A
3,AAPL,139.07,N/A,42.21,N/A,35.24,N/A,8.46,N/A,29.784121,N/A,21.709835,N/A,N/A
4,ABBV,108.57,N/A,23.6,N/A,12.77,N/A,4.74,N/A,16.132946,N/A,9.737046,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [14]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,211.440,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,241.120,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,10.02,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,27.420,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.090,N/A,14.49,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,57.030,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,29.650,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,147.450,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
194,FTI,9.900,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1772.700,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [15]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True )

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [16]:
rv_dataframe.head()

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,123.96,N/A,55.78,N/A,7.50,N/A,7.1100,N/A,33.513979,N/A,13.356296,N/A,N/A
1,AAL,16.46,N/A,-1.15,N/A,-1.73,N/A,0.3957,N/A,-6.101125,N/A,1.398429,N/A,N/A
2,AAP,159.88,N/A,23.10,N/A,2.90,N/A,1.0800,N/A,10.958001,N/A,2.510845,N/A,N/A
3,AAPL,139.07,N/A,42.21,N/A,35.24,N/A,8.4600,N/A,29.784121,N/A,21.709835,N/A,N/A
4,ABBV,108.57,N/A,23.60,N/A,12.77,N/A,4.7400,N/A,16.132946,N/A,9.737046,N/A,N/A


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [17]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}


for row in range(0, len(rv_dataframe)):
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100



#Print the entire DataFrame    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,123.96,N/A,55.78,0.865347,7.50,0.722772,7.1100,0.8,33.513979,0.843564,13.356296,0.79802,N/A
1,AAL,16.46,N/A,-1.15,0.140594,-1.73,0.0455446,0.3957,0.0277228,-6.101125,0.0277228,1.398429,0.0554455,N/A
2,AAP,159.88,N/A,23.10,0.439604,2.90,0.40198,1.0800,0.158416,10.958001,0.259406,2.510845,0.124752,N/A
3,AAPL,139.07,N/A,42.21,0.754455,35.24,0.964356,8.4600,0.848515,29.784121,0.819802,21.709835,0.946535,N/A
4,ABBV,108.57,N/A,23.60,0.453465,12.77,0.853465,4.7400,0.671287,16.132946,0.49505,9.737046,0.659406,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.97,N/A,34.10,0.669307,-4.22,0.0415842,6.2500,0.766337,24.171103,0.736634,10.313342,0.685149,N/A
501,ZBH,157.93,N/A,-216.95,0.019802,2.82,0.390099,4.6800,0.628713,27.182799,0.784158,8.037808,0.516832,N/A
502,ZBRA,389.50,N/A,45.18,0.776238,11.13,0.831683,4.8200,0.681188,30.333891,0.827723,11.555299,0.720792,N/A
503,ZION,45.38,N/A,19.15,0.372277,1.00,0.1,2.3900,0.39505,10.971564,0.261386,2.770143,0.142574,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [18]:
from statistics import mean

for row in rv_dataframe.index:
    values = [rv_dataframe.loc[row, percentileName] for percentileName in metrics.values()]
    m = mean(values)
    rv_dataframe.loc[row, "RV Score"] = m
    

## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [19]:
rv_dataframe.sort_values(["RV Score"], inplace = True, ascending=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(inplace = True, drop=True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [20]:
portfolio_size = portfolio_input()

Enter the value of your portfolio:1000000000


In [21]:
position = portfolio_size / len(rv_dataframe)
rv_dataframe["Number of Shares to Buy"] = rv_dataframe.apply(lambda x: math.floor(position / x["Price"]), axis = 1)

<ipython-input-21-ee449b56f9e7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_dataframe["Number of Shares to Buy"] = rv_dataframe.apply(lambda x: math.floor(position / x["Price"]), axis = 1)


In [22]:
rv_dataframe.head()

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,23.61,847098,5.02,0.154455,0.4436,0.049505,0.4091,0.0316832,2.488714,0.0336634,0.385624,0.0039604,0.0546535
1,AIG,38.77,515862,-6.75,0.106931,0.5284,0.0574257,0.7156,0.0633663,4.456284,0.0415842,0.704703,0.0138614,0.0566337
2,AAL,16.46,1215066,-1.15,0.140594,-1.7300,0.0455446,0.3957,0.0277228,-6.101125,0.0277228,1.398429,0.0554455,0.0594059
3,HFC,26.74,747943,-27.10,0.0673267,0.7859,0.0752475,0.3042,0.0138614,6.026941,0.0613861,2.092798,0.106931,0.0649505
4,MET,48.00,416666,7.80,0.163366,0.5897,0.0613861,0.6610,0.0554455,4.365785,0.039604,0.624307,0.0118812,0.0663366


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [23]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [24]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [25]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [26]:
writer.save()